# Earth Engine in Jupyter Notebook Demonstration

Adapted from https://developers.google.com/earth-engine/python_install-colab

### Import the API
Run the following cell to import the API into your session.

In [1]:
import ee

### Authenticate and initialise
Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Add a code cell, enter the following lines, and run the cell.

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/0QFqnSDiOAKnGl1Pk9Pb52fnuZok97Z_9AkRKOmSBy66kGWfkoApnyk



Successfully saved authorization token.


You'll be asked to authorize access to your Earth Engine account. Follow the instructions printed to the cell to complete this step.

### Test the API
Test the API by printing the elevation of Mount Everest. Note that before using the API you must initialize it. Run the following Python script in a new cell.

In [3]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


## Find data
A range of imagery including DEMs, Landsat, Sentinel and PALSAR exist. Explore options at https://developers.google.com/earth-engine/datasets/

### Filter Sentinel 1 collection

Most image processing in GEE will follow these steps:
- Find image collection to use, options can be found here: https://developers.google.com/earth-engine/datasets/
- `filterDate` to select the time period of interest
- `filterBounds` if you have some geometry to filter by, otherwise the parameters of the ineractive map will define where imagery is downloaded
- `filter` which is used in combination with more specific filters. Commonly `filer(ee.Filter.lt)` for quickly removing cloud cover. Any metaata stored in image properties can be filtered on, as an example see https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#image-properties 
- to summarize an `ee.ImageCollection` most statistics such as `median()`, `min()` and `max()` are avaliable

More information on using Sentinel 1 is avaliable here https://developers.google.com/earth-engine/sentinel1

In [4]:
# Filter image collection
imgVV = ee.ImageCollection('COPERNICUS/S1_GRD').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.eq('instrumentMode', 'IW')).select('VV')

# Create a descending mean image
desc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).filterDate('2020-01-01','2020-05-27').mean();

### Map visualisation
`ee.Image` objects can be displayed to notebook output cells, either as a static image or an interactive map.

#### Interactive map
The `folium` package can be used to display `ee.Image` objects on an interactive Leaflet map. Folium has no default method for handling tiles from Earth Engine, so one must be defined and added to the `folium.Map` module before use.

1. Set map and image variables
2. Add a method for handing Earth Engine tiles and using it to display an Sentinel 2 data to a Leaflet map.

In [5]:
# Map location in lats and lons
location=[-35.3, 149.1]
# How zoomed in the map is
zoom_start=12

# Image to use
image = desc
# Set visualization parameters.
vis_params = {
  'bands':['VV'], # check which abnds here https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#bands
  'min': -25, # pixel value
  'max': 5 # pixel value
}
display_name = 'Sentinel 1 mean'

In [6]:
# Import libraries.
import ee
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=location, zoom_start=zoom_start, height=500)

# Add the elevation model to the map object.
my_map.add_ee_layer(image, vis_params, display_name)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [ ]:
## Sentinel